In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from smooth.adam_general.core.adam import ADAM
import numpy as np
import pandas as pd

### Test 1: Global level ETS(A,N,N) - backcasting vs optimal

In [3]:
np.random.seed(33)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [4]:
model = ADAM(model='ANN', lags=[12], initial='optimal')
model.fit(ts_df)
print('ETS(A,N,N) with optimal initial:')
print(model.predict(h=12)['mean'].values)

ETS(A,N,N) with optimal initial:
[99.61957494 99.61957494 99.61957494 99.61957494 99.61957494 99.61957494
 99.61957494 99.61957494 99.61957494 99.61957494 99.61957494 99.61957494]


In [5]:
model = ADAM(model='ANN', lags=[12], initial='backcasting', n_iterations=1)
model.fit(ts_df)
print('ETS(A,N,N) with backcasting (n_iterations=1):')
print(model.predict(h=12)['mean'].values)

ETS(A,N,N) with backcasting (n_iterations=1):
[101.76341597 101.76341597 101.76341597 101.76341597 101.76341597
 101.76341597 101.76341597 101.76341597 101.76341597 101.76341597
 101.76341597 101.76341597]


In [6]:
model = ADAM(model='ANN', lags=[12], initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(A,N,N) with backcasting (n_iterations=2):')
print(model.predict(h=12)['mean'].values)

ETS(A,N,N) with backcasting (n_iterations=2):
[101.13084689 101.13084689 101.13084689 101.13084689 101.13084689
 101.13084689 101.13084689 101.13084689 101.13084689 101.13084689
 101.13084689 101.13084689]


### Test 2: Local level ETS(A,N,N)

In [7]:
np.random.seed(41)
n_points = 120
errors = np.random.normal(0, 10, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [8]:
model = ADAM(model='ANN', lags=[12], initial='optimal')
model.fit(ts_df)
print('ETS(A,N,N) with optimal initial:')
print(model.predict(h=12)['mean'].values)

ETS(A,N,N) with optimal initial:
[84.70888119 84.70888119 84.70888119 84.70888119 84.70888119 84.70888119
 84.70888119 84.70888119 84.70888119 84.70888119 84.70888119 84.70888119]


In [9]:
model = ADAM(model='ANN', lags=[12], initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(A,N,N) with backcasting (n_iterations=2):')
print(model.predict(h=12)['mean'].values)

ETS(A,N,N) with backcasting (n_iterations=2):
[84.71061589 84.71061589 84.71061589 84.71061589 84.71061589 84.71061589
 84.71061589 84.71061589 84.71061589 84.71061589 84.71061589 84.71061589]


### Test 3: Local trend ETS(A,A,N)

In [10]:
np.random.seed(42)
n_points = 120
errors = np.random.normal(0, 10, n_points)
trend = np.random.normal(0.5, 2, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [11]:
model = ADAM(model='AAN', lags=[12], persistence=[0.1, 0.02], initial='optimal')
model.fit(ts_df)
print('ETS(A,A,N) with optimal initial:')
print(model.predict(h=12)['mean'].values)

ETS(A,A,N) with optimal initial:
[165.25552478 165.7639546  166.27238442 166.78081423 167.28924405
 167.79767387 168.30610369 168.8145335  169.32296332 169.83139314
 170.33982296 170.84825277]


In [12]:
model = ADAM(model='AAN', lags=[12], persistence=[0.1, 0.02], initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(A,A,N) with backcasting (n_iterations=2):')
print(model.predict(h=12)['mean'].values)


ETS(A,A,N) with backcasting (n_iterations=2):
[165.25637326 165.76469417 166.27301508 166.781336   167.28965691
 167.79797782 168.30629874 168.81461965 169.32294056 169.83126147
 170.33958239 170.8479033 ]


### Test 4: Damped trend ETS(A,Ad,N)

In [13]:
model = ADAM(model='AAdN', lags=[12], initial='optimal')
model.fit(ts_df)
print('ETS(A,Ad,N) with optimal initial:')
print(model.adam_estimated['B'])
print(model.predict(h=12)['mean'].values)

ETS(A,Ad,N) with optimal initial:
[ 1.68275229e-01  1.04248642e-02  9.86301308e-01  1.04774858e+02
 -3.80921118e-01]
[164.87482292 165.48069044 166.07825836 166.66764039 167.24894865
 167.82229375 168.38778477 168.9455293  169.49563347 170.03820192
 170.5733379  171.10114321]


In [14]:
model = ADAM(model='AAdN', lags=[12], initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(A,Ad,N) with backcasting (n_iterations=2):')
print(model.adam_estimated['B'])
print(model.predict(h=12)['mean'].values)

ETS(A,Ad,N) with backcasting (n_iterations=2):
[0.17307159 0.00969924 0.99006236]
[165.07034377 165.70540983 166.33416483 166.95667148 167.57299189
 168.18318752 168.78731925 169.38544733 169.97763142 170.56393061
 171.14440336 171.71910758]


### Test 5: Multiplicative error ETS(M,N,N)

In [15]:
model = ADAM(model='MNN', lags=[12], distribution='dnorm', initial='optimal')
model.fit(ts_df)
print('ETS(M,N,N) with optimal initial:')
print(model.predict(h=12)['mean'].values)

ETS(M,N,N) with optimal initial:
[163.03569195 163.03569195 163.03569195 163.03569195 163.03569195
 163.03569195 163.03569195 163.03569195 163.03569195 163.03569195
 163.03569195 163.03569195]


In [16]:
model = ADAM(model='MNN', lags=[12], distribution='dnorm', initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(M,N,N) with backcasting (n_iterations=2):')
print(model.predict(h=12)['mean'].values)

ETS(M,N,N) with backcasting (n_iterations=2):
[163.0335895 163.0335895 163.0335895 163.0335895 163.0335895 163.0335895
 163.0335895 163.0335895 163.0335895 163.0335895 163.0335895 163.0335895]


### Test 6: Multiplicative with additive trend ETS(M,A,N)

In [17]:
model = ADAM(model='MAN', lags=[12], persistence=[0.1, 0.02], distribution='dnorm', initial='optimal')
model.fit(ts_df)
print('ETS(M,A,N) with optimal initial:')
print(model.predict(h=12)['mean'].values)

ETS(M,A,N) with optimal initial:
[165.25625383 165.76454565 166.27283746 166.78112928 167.28942109
 167.79771291 168.30600473 168.81429654 169.32258836 169.83088017
 170.33917199 170.8474638 ]


In [18]:
model = ADAM(model='MAN', lags=[12], persistence=[0.1, 0.02], distribution='dnorm', initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(M,A,N) with backcasting (n_iterations=2):')
print(model.predict(h=12)['mean'].values)


ETS(M,A,N) with backcasting (n_iterations=2):
[165.25637326 165.76469417 166.27301508 166.781336   167.28965691
 167.79797782 168.30629874 168.81461965 169.32294056 169.83126147
 170.33958239 170.8479033 ]


### Test 7: Multiplicative damped trend ETS(M,Ad,N)

In [19]:
model = ADAM(model='MAdN', lags=[12], distribution='dnorm', initial='optimal')
model.fit(ts_df)
print('ETS(M,Ad,N) with optimal initial:')
print(model.adam_estimated['B'])
print(model.predict(h=12)['mean'].values)

ETS(M,Ad,N) with optimal initial:
[ 1.49690928e-01  1.32751821e-02  9.67541206e-01  1.05756209e+02
 -4.62346860e-01]
[164.14318554 164.65545692 165.15110059 165.63065627 166.09464614
 166.54357547 166.97793309 167.39819198 167.80480978 168.19822925
 168.57887881 168.94717293]


In [20]:
model = ADAM(model='MAdN', lags=[12], distribution='dnorm', initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(M,Ad,N) with backcasting (n_iterations=2):')
print(model.adam_estimated['B'])
print(model.predict(h=12)['mean'].values)

ETS(M,Ad,N) with backcasting (n_iterations=2):
[0.15600482 0.01266795 0.97163619]
[164.29106338 164.81745719 165.32892047 165.8258767  166.30873736
 166.77790225 167.23375984 167.67668757 168.10705219 168.52521002
 168.93150731 169.32628046]


### Test 8: Seasonal data ETS(A,A,A)

In [21]:
np.random.seed(42)
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))
trend = np.random.normal(0.5, 2, n_points)
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [22]:
model = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='optimal')
model.fit(ts_df)
print('ETS(A,A,A) with optimal initial:')
print(model.adam_estimated['B'])
print(model.predict(h=12)['mean'].values)

ETS(A,A,A) with optimal initial:
[ 2.16181104e-02  1.57907500e-02  5.85935466e-04  2.21859547e+02
 -7.47106191e-01 -2.78979319e+01 -1.11063434e+01 -1.87719047e-01
  3.39782529e+01 -4.30939382e+01  1.16870694e+00 -1.50195038e+01
 -2.03612866e+01  7.23885535e+01  9.66532310e+00 -3.49830521e+01]
[139.42525781 157.07965048 168.90171789 203.93720125 127.74325507
 172.84832684 157.58226369 153.12013456 246.738613   184.9105097
 141.14659227 212.4595497 ]


In [23]:
model = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(A,A,A) with backcasting (n_iterations=2):')
print(model.adam_estimated['B'])
print(model.predict(h=12)['mean'].values)

ETS(A,A,A) with backcasting (n_iterations=2):
[0.04116081 0.01328189 0.        ]
[140.29169515 154.96116382 166.75348535 199.78305367 126.85593695
 165.60471231 155.19548222 149.66749282 242.20686221 184.21656916
 138.15798761 208.38579437]


### Test 9: Seasonal damped ETS(A,Ad,A)

In [24]:
model = ADAM(model='AAdA', lags=[12], distribution='dnorm', initial='optimal')
model.fit(ts_df)
print('ETS(A,Ad,A) with optimal initial:')
print(model.predict(h=12)['mean'].values)

ETS(A,Ad,A) with optimal initial:
[143.36857734 159.17746839 170.619509   206.31180659 128.70023417
 165.90543049 158.7444117  152.92724171 245.09866352 184.76173526
 139.86023558 212.09605534]


In [25]:
model = ADAM(model='AAdA', lags=[12], distribution='dnorm', initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(A,Ad,A) with backcasting (n_iterations=2):')
print(model.predict(h=12)['mean'].values)

ETS(A,Ad,A) with backcasting (n_iterations=2):
[140.96473562 155.51219138 167.17651269 200.07216766 127.00529766
 165.60855215 155.04810502 149.36327295 241.74024373 183.58206491
 137.35017839 207.3993281 ]


### Test 10: Multiplicative seasonal ETS(M,A,M)

In [26]:
model = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='optimal')
model.fit(ts_df)
print('ETS(M,A,M) with optimal initial:')
print(model.adam_estimated['B'])
print(model.predict(h=12)['mean'].values)

ETS(M,A,M) with optimal initial:
[1.48159059e-01 1.42680375e-02 7.38461996e-04 2.03782026e+02
 6.27206481e-01 8.65002472e-01 9.46165077e-01 1.03216495e+00
 1.20151786e+00 7.70686649e-01 9.88553426e-01 9.26662454e-01
 9.07359777e-01 1.41730701e+00 1.06717115e+00 8.23421502e-01]
[146.54871673 160.51279419 175.33296921 204.39075531 131.27542559
 168.60735789 158.26980976 155.16627803 242.71101665 183.00177292
 141.38705469 212.3148902 ]


In [27]:
model = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(M,A,M) with backcasting (n_iterations=2):')
print(model.adam_estimated['B'])
print(model.predict(h=12)['mean'].values)

ETS(M,A,M) with backcasting (n_iterations=2):
[3.72037153e-02 1.23201909e-02 2.26162817e-07]
[142.06377405 155.21117349 166.09280697 197.85664922 128.3185695
 165.09370023 155.32968337 149.06001387 237.40819796 182.65040118
 138.48414814 205.52244382]


    ### Test 11: Multiplicative seasonal damped ETS(M,Ad,M)

In [28]:
model = ADAM(model='MAdM', lags=[12], distribution='dnorm', initial='optimal')
model.fit(ts_df)
print('ETS(M,Ad,M) with optimal initial:')
print(model.predict(h=12)['mean'].values)

ETS(M,Ad,M) with optimal initial:
[145.29181408 161.00470827 173.90703806 202.67886554 129.83804209
 167.28482152 157.87264635 154.07757475 241.79766913 181.03009805
 139.81140002 210.1556838 ]


In [29]:
model = ADAM(model='MAdM', lags=[12], distribution='dnorm', initial='backcasting', n_iterations=2)
model.fit(ts_df)
print('ETS(M,Ad,M) with backcasting (n_iterations=2):')
print(model.predict(h=12)['mean'].values)

ETS(M,Ad,M) with backcasting (n_iterations=2):
[143.23815915 156.27631904 166.99697194 198.65005052 128.64733846
 165.2754287  155.27164404 148.78265101 236.61039116 181.76050293
 137.5985553  203.89287636]
